# Imports

In [1]:
import pandas as pd
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td

import gc
from pathlib import Path
import typing as tp
from tqdm.autonotebook import tqdm, trange
import wandb
import warnings
import json
import optuna
from pprint import pprint
from itertools import product
from typing import Literal

from metrics import mean_average_precision, precision_recall_at_k
from early_stopping import EarlyStopping


# torch.multiprocessing.set_start_method('fork', force=True)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
torch.manual_seed(31337)

/tmp/ipykernel_1175670/2129261885.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Load data

In [2]:
# folder = Path("/content/")
folder = Path("data/data_kion")
users_df = pd.read_parquet(folder / "users_final.parquet")
items_df = pd.read_parquet(folder / "items_final.parquet")
train_data = pd.read_parquet(folder / "train_val_triplets.parquet")
val_data   = pd.read_parquet(folder / "test_pos.parquet")

In [3]:
NO_MOVIE = int(items_df.index.max()) + 1 # id bigger than any item id
PAD_SIZE = 30  # users' interactions are padded to have the same number of items
MOVIE_IDS_DTYPE = torch.int16
USER_IDS_DTYPE = torch.int32
MAIN_DTYPE = torch.float32

ITEMS_NUM_GENRE_FEATURES = 29
ITEMS_NUM_CAT_FEATURES = 4
NO_MOVIE

15963

In [4]:
most_popular_films = torch.tensor(train_data['film_pos'].value_counts()[:200].index.tolist(), dtype=MOVIE_IDS_DTYPE)
most_popular_films

tensor([13387,  9393,  3599, 14769, 10072,   139,  8333,  4000,  1784,  7309,
        10839,  4337,  4300, 13933,  7159, 14197,  6576, 12539,  4279,  8852,
         6855, 15616,  7364,   636,  3067,  9651, 13821, 14235, 14927,  5496,
        12525, 12562,  4571,  3791,  3648, 11362, 11745,  7560,  5462, 12057,
         1761, 10908,  1247, 11338,  6394, 10388,  1728, 12295, 10096, 13960,
        13434,  9724, 14390,  1096,  7320, 10377, 12390,  4313,  5243, 10553,
        15716,  5232, 13863,   825, 11918,  6174,  3383, 15806,  6227,  6961,
        12781,  2844,  2161,  5940, 11888, 15538, 15369, 11333,  8315,  2958,
         5556, 12700,  2144, 11564,  4528,  8148,   283, 14738,  2625,  1506,
         8673,  8140,   355, 12177, 10068,  4550,  5112,  8512,  2222,  8895,
        13076, 14831,  1404,  8493,  2981,  8875,  5075,  4534, 11340, 10945,
         4106,  2565, 15195, 13969,  1373,  1855, 13540,  5601,  9764,  8082,
         7056,  9560, 12020,  1566, 12509,  9504,  4084, 12321, 

# Dataset

In [5]:
def pad_with_specific_value(tensor, target_length, pad_value):
    """`tensor` is considered to have only unique values"""
    shuffled = tensor[torch.randperm(len(tensor))[:target_length]]
    cur_length = len(shuffled)
    if cur_length < target_length:
        padding = torch.full((target_length - cur_length,), pad_value, dtype=shuffled.dtype)
        return torch.cat([shuffled, padding])
    else:
        return shuffled

def group_by_user(triplets: pd.DataFrame, column: Literal['film_pos', 'film_neg'] = 'film_pos') -> pd.DataFrame:
    """Group interactions by user and convert to DataFrame with column 'interactions' - Tensors of unique film IDs"""
    groupped_users = triplets.groupby('user_id').apply(lambda x: x[column].tolist())
    groupped_users = pd.DataFrame({'interactions': groupped_users.values}, index=groupped_users.index)
    groupped_users['interactions'] = groupped_users['interactions'].apply(lambda x: torch.unique(torch.tensor(x, dtype=MOVIE_IDS_DTYPE)))
    return groupped_users

In [6]:
def collate_fn(data: list[tuple]):
    return data

class BaseDSSMDataset(torch.utils.data.Dataset):
    def __init__(self):
        super().__init__()
        self.type = None
        self.seen_items = {}

class TrainDSSMDataset(BaseDSSMDataset):
    def __init__(self, triplets: pd.DataFrame):
        super().__init__()
        self.type = 'train'
        self.triplets = triplets
        self.grouped_pos_users_interactions = group_by_user(triplets, column='film_pos')  # just like padded users but not padded
        self.padded_users = self.grouped_pos_users_interactions.copy(deep=True)
        self.padded_users['interactions'] = self.padded_users['interactions'].apply(lambda x: pad_with_specific_value(x, PAD_SIZE, NO_MOVIE).to(dtype=torch.int32))  # int32 required by EmbeddingBag

        self.grouped_neg_users_interactions = group_by_user(triplets, column='film_neg')  # all negative interactions grouped by users into tensors
        self.all_users = torch.tensor(self.grouped_pos_users_interactions.index.tolist(), dtype=USER_IDS_DTYPE)  # all unique users from triplets

    def __getitem__(self, index: int):
        cur_triplet = self.triplets.iloc[index]
        user_id = cur_triplet['user_id']

        user_info = torch.tensor(users_df.loc[user_id].values, dtype=MAIN_DTYPE)
        user_interactions = self.padded_users.loc[user_id]['interactions']
        pos_films_features = torch.tensor(items_df.loc[cur_triplet['film_pos']].values, dtype=MAIN_DTYPE)
        neg_films_features = torch.tensor(items_df.loc[cur_triplet['film_neg']].values, dtype=MAIN_DTYPE)
        pos_ids = self.grouped_pos_users_interactions.loc[user_id]['interactions']

        return user_id, user_interactions, user_info, pos_films_features, neg_films_features, pos_ids

    def __getitems__(self, index: tp.Sequence[int]):
        cur_triplets = self.triplets.iloc[index]
        user_ids = cur_triplets['user_id']

        user_info = torch.tensor(users_df.loc[user_ids].values, dtype=MAIN_DTYPE)
        user_interactions = torch.stack(self.padded_users.loc[user_ids]['interactions'].tolist())
        pos_films_features = torch.tensor(items_df.loc[cur_triplets['film_pos']].values, dtype=MAIN_DTYPE)
        neg_films_features = torch.tensor(items_df.loc[cur_triplets['film_neg']].values, dtype=MAIN_DTYPE)
        pos_ids = self.grouped_pos_users_interactions.loc[user_ids]['interactions'].tolist()

        return user_ids, user_interactions, user_info, pos_films_features, neg_films_features, pos_ids

    def get_user_data(self, user_id: int) -> tuple[Tensor, Tensor]:
        user_info = torch.tensor(users_df.loc[user_id].values, dtype=MAIN_DTYPE)
        if user_id in self.padded_users.index:
            user_interactions = self.padded_users.loc[user_id]['interactions']
        else:
            user_interactions = torch.full((PAD_SIZE,), NO_MOVIE, dtype=torch.int32)

        return user_interactions, user_info

    def get_users_data(self, user_ids: int) -> tuple[Tensor, Tensor]:
        user_info = torch.tensor(users_df.loc[user_ids].values, dtype=MAIN_DTYPE)

        inters = []
        for uid in user_ids:
            if uid in self.padded_users.index:
                inter = self.padded_users.loc[uid, 'interactions']
            else:
                inter = torch.full(
                    (PAD_SIZE,),
                    NO_MOVIE,
                    dtype=torch.int32
                )
            inters.append(inter)

        user_interactions = torch.stack(inters, dim=0)

        return user_interactions, user_info

    def __len__(self):
        return len(self.triplets)

class EvalDSSMDataset(BaseDSSMDataset):
    def __init__(self, pos_df: pd.DataFrame):
        super().__init__()
        self.type = 'eval'
        self.all_users = pos_df['user_id'].unique().astype(int)
        self.grouped_pos_users_interactions = group_by_user(pos_df, column='film_pos')
        # self.seen_items = train_triplets.grouped_pos_users_interactions | train_triplets.grouped_neg_users_interactions  # seen items are all positive and negative interactions from train triplets

    def __getitem__(self, index: int):
        user = int(self.all_users[index])
        pos  = int(self.grouped_pos_users_interactions[user]['interactions'])
        return user, pos

    def __getitems__(self, index: tp.Sequence[int]):
        users = self.all_users[index]
        pos   = self.grouped_pos_users_interactions.loc[users]['interactions'].tolist()
        return users, pos

    def get_all_users(self) -> Tensor:
        """
        Get IDs of all users, that have positive interactions in this dataset

        Returns:
            users (Tensor, dtype=torch.int32) : All user IDs
        """
        return self.all_users

    def __len__(self):
        return len(self.all_users)


# Model

In [7]:
def _make_residual_mlp(dim: int, hidden: int, dropout: float):
    return nn.Sequential(
        nn.Linear(dim, hidden),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden, dim),
        nn.Dropout(dropout),
        nn.LayerNorm(dim)
    )


class ItemNet(nn.Module):
    def __init__(self,
                 dim_embedding: int,
                 dim_input: int,
                 dim_hidden: int = 96,
                 dropout: float = 0.2,
                 activation: tp.Callable[[Tensor], Tensor] = nn.ReLU()
                 ) -> None:
        super().__init__()
        self.cat_embedding = nn.Linear(ITEMS_NUM_CAT_FEATURES, dim_hidden)
        self.genre_embedding = nn.Linear(ITEMS_NUM_GENRE_FEATURES, dim_hidden)
        self.country_embedding = nn.Linear(dim_input - ITEMS_NUM_CAT_FEATURES - ITEMS_NUM_GENRE_FEATURES, dim_hidden)

        # self.dense_block = nn.Sequential(
        #     nn.Linear(dim_hidden * 3, dim_hidden * 3),
        #     activation,
        # )
        self.res1 = _make_residual_mlp(dim_hidden * 3, dim_hidden * 2, dropout)
        self.res2 = _make_residual_mlp(dim_hidden * 3, dim_hidden * 2, dropout)

        self.output_layer = nn.Linear(dim_hidden * 3, dim_embedding, bias=False)
        self.norm = nn.LayerNorm(dim_embedding)

    def forward(self, item_features: Tensor) -> Tensor:
        cat_features = item_features[:, :ITEMS_NUM_CAT_FEATURES]
        genre_features = item_features[:, ITEMS_NUM_CAT_FEATURES:ITEMS_NUM_CAT_FEATURES + ITEMS_NUM_GENRE_FEATURES]
        country_features = item_features[:, ITEMS_NUM_CAT_FEATURES + ITEMS_NUM_GENRE_FEATURES:]

        cat_emb = self.cat_embedding(cat_features)
        genre_emb = self.genre_embedding(genre_features)
        country_emb = self.country_embedding(country_features)

        h = torch.cat([cat_emb, genre_emb, country_emb], dim=1)

        # h = self.dense_block(h)
        h = self.res1(h) + h
        h = self.res2(h) + h

        output = self.output_layer(h)
        return self.norm(output)


class UserNet(nn.Module):
    def __init__(self,
                 dim_embedding: int,
                 num_items: int,
                 dim_user_features: int,
                 max_seq_len: int = 50,
                 n_heads: int = 4,
                 activation: tp.Callable[[Tensor], Tensor] = nn.ReLU(),
                 dropout: float = 0.2
                 ) -> None:                              # | +1 for the NO_MOVIE element
        super().__init__()                               # V
        self.interactions_embeddings = nn.EmbeddingBag(num_items + 1, dim_embedding, padding_idx=num_items)
        # self.info_embedding = nn.Sequential(
        #     nn.Linear(dim_user_features, int(dim_embedding // 2)),
        #     nn.ReLU(),
        #     nn.Linear(int(dim_embedding // 2), dim_embedding),
        #     activation
        # )

        # self.item_embedding = nn.Embedding(num_items + 1, dim_embedding, padding_idx=num_items)
        # self.pos_embedding = nn.Embedding(max_seq_len, dim_embedding)
        # self.transformer = nn.TransformerEncoderLayer(
        #     d_model=dim_embedding,
        #     nhead=n_heads,
        #     dim_feedforward=dim_embedding * 2,
        #     dropout=dropout,
        #     activation='relu'
        # )

        self.dense_layer = nn.Sequential(
            nn.Linear(dim_user_features, int(dim_embedding // 2)),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(int(dim_embedding // 2), dim_embedding),
            activation
        )
        self.output_layer = nn.Linear(2*dim_embedding, dim_embedding, bias=False)
        self.norm = nn.LayerNorm(dim_embedding)
        self.num_items = num_items
        self.dim_user_features = dim_user_features

    def forward(self, user_interactions: Tensor, user_info: Tensor) -> Tensor:
        interactions_emb = self.interactions_embeddings(user_interactions)
        # info_emb = self.info_embedding(user_info.float())

        # B, L = user_interactions.size()
        # pos = torch.arange(L, device=user_interactions.device).unsqueeze(0).expand(B, -1)
        # e = self.item_embedding(user_interactions) + self.pos_embedding(pos)
        # # Transformer expects [L, B, D]
        # h = self.transformer(e.permute(1, 0, 2), src_key_padding_mask=(user_interactions == self.item_embedding.padding_idx))
        # h = h.permute(1, 0, 2) 
        # interactions_emb = h.mean(dim=1)

        features = self.dense_layer(user_info)
        x = torch.cat([interactions_emb, features], dim=1)
        output = self.output_layer(x)
        return self.norm(output)

In [8]:
class DSSM(nn.Module):
    def __init__(self,
                 dim_item_features: int,
                 dim_user_features: int,
                 num_items: int,
                 embedding_dim: int = 100,
                 ) -> None:
        super().__init__()
        self.item_net = ItemNet(embedding_dim, dim_item_features)
        self.user_net = UserNet(embedding_dim, num_items, dim_user_features)

    def forward(self,
                user_intercations: Tensor,
                user_info: Tensor,
                item_features_pos: Tensor,
                item_features_neg: Tensor,
                ) -> tuple[Tensor, Tensor, Tensor]:
        """Returns embeddings of users, positive items and negative items"""
        anchor = self.user_net(user_intercations, user_info)
        pos = self.item_net(item_features_pos)
        neg = self.item_net(item_features_neg)

        return anchor, pos, neg

    def encode_user(self, user_interactions: Tensor, user_info: Tensor) -> Tensor:
        return self.user_net(user_interactions, user_info)

    def encode_item(self, item_features: Tensor) -> Tensor:
        return self.item_net(item_features)

# Training

## Configure training

In [9]:
print('Get train dataset')
train_dataset = TrainDSSMDataset(train_data)
print('Get validation dataset')
val_dataset = EvalDSSMDataset(val_data)

Get train dataset
Get validation dataset


In [10]:
# Hyperparams
EPOCHS = 50
BATCH_SIZE = 1024 * 4
NUM_WORKERS = 0
METRICS_N_JOBS = 12
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

EMBEDDING_DIM = 1024
LR = 5e-3
WEIGHT_DECAY = 1e-5
TRIPLET_LOSS_MARGIN = 0.4
EXPERIMENT_NAME = 'New_arch_no_transformer_train+val_choose_top_200_popular'
K = 10
LOG_TO_WANDB = True

train_dataloader = td.DataLoader(train_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_dataloader = td.DataLoader(val_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

model = DSSM(dim_item_features=items_df.shape[1],
             dim_user_features=users_df.shape[1],
             num_items=NO_MOVIE,
             embedding_dim=EMBEDDING_DIM
             ).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    patience=5,
    mode='max',
    threshold=2e-3,
    threshold_mode='abs',
    factor=torch.sqrt(torch.tensor(0.1))
)
early_stopping = EarlyStopping(
    patience=11,
    mode='max',
    threshold=2e-3,
    threshold_mode='abs',
    verbose=True,
    trace_func=tqdm.write,
    path=f'model_weights/{EXPERIMENT_NAME}.pt'
)

## Training loop

In [11]:
if LOG_TO_WANDB:
    entity = "xenz5240-higher-school-of-economics"
    wandb.init(entity=entity, project='reelsrec-dssm', name=EXPERIMENT_NAME)

    wandb.config.update({
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "embedding_dim": EMBEDDING_DIM,
        "learning_rate": LR,
        "weight_decay": WEIGHT_DECAY,
        "triplet_loss_margin": TRIPLET_LOSS_MARGIN,
        "device": DEVICE
    })

for epoch in trange(EPOCHS, position=0, desc='Training', unit='epoch'):
    model.train()
    epoch_losses = []
    batch_times = []
    for step, batch in enumerate(tqdm(train_dataloader, position=1, desc=f'Epoch {epoch + 1}/{EPOCHS}', unit='batch')):
        _, user_inters, user_info, pos_features, neg_features, _ = batch
        optimizer.zero_grad()
        batch_users_embs, batch_positive_films_embs, batch_negative_films_embs = model(user_inters.to(DEVICE),
                                                                                       user_info.to(DEVICE),
                                                                                       pos_features.to(DEVICE),
                                                                                       neg_features.to(DEVICE))
        loss = F.triplet_margin_loss(batch_users_embs, batch_positive_films_embs, batch_negative_films_embs, margin=TRIPLET_LOSS_MARGIN)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

        if LOG_TO_WANDB and step % 20 == 0:
            wandb.log({"train/loss": loss.item(),
                       "epoch": epoch + 1,
                       "step": step})

    mean_train_loss = sum(epoch_losses) / len(epoch_losses)
    if LOG_TO_WANDB:
        wandb.log({"train/epoch_loss": mean_train_loss, "epoch": epoch + 1})

    # Evaluation
    model.eval()
    all_predictions = []
    all_references = []
    # Embed all items
    items_df.sort_index(inplace=True)  # Ensure items are sorted by their IDs
    item_features = torch.tensor(items_df.values, dtype=MAIN_DTYPE, device=DEVICE)
    EMPTY_WATCH_HISTORY = torch.empty(0, dtype=MOVIE_IDS_DTYPE)
    with torch.no_grad():
        all_items_embeds = model.encode_item(item_features)

        for step, batch in enumerate(tqdm(val_dataloader,
                                          desc=f'Validation {epoch + 1}/{EPOCHS}',
                                          leave=False,
                                          position=1,
                                          unit='batch')):
            user_ids, positive_interactions = batch
            user_inters, user_info = train_dataset.get_users_data(user_ids)
            user_embs = model.encode_user(user_inters.to(DEVICE), user_info.to(DEVICE))

            # Build distance matrix
            distance_matrix = torch.cdist(user_embs, all_items_embeds, p=2.0)
            val_recommendations = distance_matrix.argsort(dim=1, descending=False).to(dtype=MOVIE_IDS_DTYPE, device='cpu')  # (batch_users x all_items)

            # Filter watched movies from recommendations and select only the most popular ones
            # Get all films each user has interacted with (from both film_pos and film_neg)
            user_interacted_films: dict[int, Tensor] = (
                train_dataset.triplets[train_dataset.triplets['user_id'].isin(user_ids.tolist())]
                .groupby('user_id')[['film_pos', 'film_neg']]
                .apply(lambda x: torch.tensor(pd.unique(x.values.ravel('K')).tolist(), dtype=MOVIE_IDS_DTYPE))
            ).to_dict()

            # For each user remove watched films from their recommendations
            filtered_val_recommendations = [None] * len(user_ids)
            for i, (user_ids, recommendations) in tqdm(enumerate(zip(user_ids, val_recommendations)),
                                                        desc='Filtering recommendations',
                                                        total=len(user_ids),
                                                        leave=False,
                                                        position=2,
                                                        unit='user'):
                watched_films = user_interacted_films.get(user_ids, EMPTY_WATCH_HISTORY)
                filtered_val_recommendations[i] = recommendations[torch.isin(recommendations, watched_films, invert=True) & torch.isin(recommendations, most_popular_films)]

            all_predictions.extend(filtered_val_recommendations)
            all_references.extend(positive_interactions)

    precision_at_k_score, recall_at_k_score = precision_recall_at_k(y_true=all_references, y_pred=all_predictions, k=K, n_jobs=METRICS_N_JOBS)
    f1_at_k_score = 2 * (precision_at_k_score * recall_at_k_score) / (precision_at_k_score + recall_at_k_score)
    map_at_10_score = mean_average_precision(y_true=all_references, y_pred=all_predictions, k=10, n_jobs=METRICS_N_JOBS)
    # map_score = mean_average_precision_full(y_true=all_references, y_pred=all_predictions, n_jobs=METRICS_N_JOBS)

    tqdm.write(f'Epoch {epoch + 1} | Precision@{K}: {precision_at_k_score:.5f} | Recall@{K}: {recall_at_k_score:.5f} | F1@{K}: {f1_at_k_score:.5f} | mAP@10: {map_at_10_score:.5f}')

    del all_predictions, all_references, filtered_val_recommendations, positive_interactions, all_items_embeds
    gc.collect()
    torch.cuda.empty_cache()

    if LOG_TO_WANDB:
        wandb.log({
            f"val/precision@{K}": precision_at_k_score,
            f"val/recall@{K}": recall_at_k_score,
            f"val/f1@{K}": f1_at_k_score,
            f"val/mAP@10": map_at_10_score,
            "train/lr": optimizer.param_groups[0]['lr'],
            "epoch": epoch + 1,
        })

    lr_scheduler.step(f1_at_k_score)
    if early_stopping(f1_at_k_score, model):
        break

wandb: Currently logged in as: sergfedchin (xenz5240-higher-school-of-economics) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Training:   0%|          | 0/50 [00:00<?, ?epoch/s]

Epoch 1/50:   0%|          | 0/1098 [00:00<?, ?batch/s]

Validation 1/50:   0%|          | 0/11 [00:00<?, ?batch/s]

Filtering recommendations:   0%|          | 0/4096 [00:00<?, ?user/s]

...

Filtering recommendations:   0%|          | 0/4096 [00:00<?, ?user/s]

Filtering recommendations:   0%|          | 0/4096 [00:00<?, ?user/s]

Filtering recommendations:   0%|          | 0/4096 [00:00<?, ?user/s]

Filtering recommendations:   0%|          | 0/3125 [00:00<?, ?user/s]

Epoch 12 | Precision@10: 0.01310 | Recall@10: 0.05277 | F1@10: 0.02099 | mAP@10: 0.02484
EarlyStopping counter: 11 out of 11


# Overnight tries

In [ ]:
def train_model(epochs: int = 50, embedding_dim: int = 192, lr: float = 1e-4, weight_decay: float = 1e-5, triplet_loss_margin: float = 0.4, wandb_id: int | str = 0):
    # Hyperparams
    EPOCHS = epochs
    BATCH_SIZE = 1024 * 16
    NUM_WORKERS = 0
    DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

    EMBEDDING_DIM = embedding_dim
    LR = lr
    WEIGHT_DECAY = weight_decay
    TRIPLET_LOSS_MARGIN = triplet_loss_margin
    EXPERIMENT_NAME = f'improved_validation_{wandb_id}'
    K = 100
    LOG_TO_WANDB = True

    train_dataloader = td.DataLoader(train_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    val_dataloader = td.DataLoader(val_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    model = DSSM(dim_item_features=items_df.shape[1],
                dim_user_features=users_df.shape[1],
                num_items=NO_MOVIE,
                embedding_dim=EMBEDDING_DIM
                ).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    # lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, threshold=3e-3, threshold_mode='abs')
    # lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #     optimizer,
    #     patience=3,
    #     threshold=1e-3,
    #     threshold_mode='abs'
    # )
    # early_stopping = EarlyStopping(
        # patience=7,
        # threshold=1e-3,
        # threshold_mode='rel',
        # verbose=True,
        # trace_func=tqdm.write,
        # path=f'model_weights/{EXPERIMENT_NAME}.ckpt'
    # )
    if LOG_TO_WANDB:
        entity = "xenz5240-higher-school-of-economics"
        wandb.init(entity=entity, project='reelsrec-dssm', name=EXPERIMENT_NAME)

        wandb.config.update({
            "epochs": EPOCHS,
            "batch_size": BATCH_SIZE,
            "embedding_dim": EMBEDDING_DIM,
            "learning_rate": LR,
            "weight_decay": WEIGHT_DECAY,
            "triplet_loss_margin": TRIPLET_LOSS_MARGIN,
            "device": DEVICE
        })
    results = {'mAP': [], 'recall': []}
    for epoch in trange(EPOCHS, position=0, desc='Training', unit='epoch'):
        model.train()
        epoch_losses = []
        for step, batch in enumerate(tqdm(train_dataloader, position=1, desc=f'Epoch {epoch + 1}/{EPOCHS}', unit='batch')):
            _, user_inters, user_info, pos_features, neg_features, _ = batch
            optimizer.zero_grad()
            batch_users_embs, batch_positive_films_embs, batch_negative_films_embs = model(user_inters.to(DEVICE),
                                                                                        user_info.to(DEVICE),
                                                                                        pos_features.to(DEVICE),
                                                                                        neg_features.to(DEVICE))
            loss = F.triplet_margin_loss(batch_users_embs, batch_positive_films_embs, batch_negative_films_embs, margin=TRIPLET_LOSS_MARGIN)
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())

            if LOG_TO_WANDB and step % 20 == 0:
                wandb.log({"train/loss": loss.item(),
                        "epoch": epoch + 1,
                        "step": step})

        mean_train_loss = sum(epoch_losses) / len(epoch_losses)
        if LOG_TO_WANDB:
            wandb.log({"train/epoch_loss": mean_train_loss, "epoch": epoch + 1})

        # Evaluation
        model.eval()
        all_predictions = []
        all_references = []
        # Embed all items
        items_df.sort_index(inplace=True)  # Ensure items are sorted by their IDs
        item_features = torch.tensor(items_df.values, dtype=MAIN_DTYPE, device=DEVICE)
        EMPTY_WATCH_HISTORY = torch.empty(0, dtype=MOVIE_IDS_DTYPE)
        with torch.no_grad():
            all_items_embeds = model.encode_item(item_features)

            for step, batch in enumerate(tqdm(val_dataloader, position=1, desc=f'Validation {epoch + 1}/{EPOCHS}', unit='batch')):
                user_ids, positive_interactions = batch
                user_inters, user_info = train_dataset.get_users_data(user_ids)
                user_embs = model.encode_user(user_inters.to(DEVICE), user_info.to(DEVICE))

                # строим матрицу расстояний
                distance_matrix = torch.cdist(user_embs, all_items_embeds, p=2.0)
                val_recommendations = distance_matrix.argsort(dim=1, descending=False).to(dtype=MOVIE_IDS_DTYPE, device='cpu')  # (batch_users x all_items)

                # фильтруем рекомендации, убирая просмотренные фильмы
                # Get all films each user has interacted with (from both film_pos and film_neg)
                user_interacted_films: dict[int, Tensor] = (train_dataset.triplets[train_dataset.triplets['user_id'].isin(user_ids.tolist())].groupby('user_id')[['film_pos', 'film_neg']].apply(lambda x: torch.tensor(pd.unique(x.values.ravel('K')).tolist(), dtype=MOVIE_IDS_DTYPE))).to_dict()

                # For each user remove watched films from their recommendations
                filtered_val_recommendations = [None] * len(user_ids)
                for i, (user_ids, recommendations) in tqdm(enumerate(zip(user_ids, val_recommendations)), desc='Filtering recommendations', total=len(user_ids), leave=False):
                    watched_films = user_interacted_films.get(user_ids, EMPTY_WATCH_HISTORY)
                    filtered_val_recommendations[i] = recommendations[torch.isin(recommendations, watched_films, invert=True)]

                all_predictions.extend(filtered_val_recommendations)
                all_references.extend(positive_interactions)

        map_score = mean_average_precision(y_true=all_references, y_pred=all_predictions, n_jobs=8)
        results['mAP'].append(map_score)
        recall_score = float(recall_at_k(y_true=all_references, y_pred=all_predictions, k=K, n_jobs=8))
        results['recall'].append(recall_score)
        tqdm.write(f'Recall@{K}: {recall_score:.5f} | mAP: {map_score:.5f}')

        del all_predictions, all_references, filtered_val_recommendations, positive_interactions, all_items_embeds
        gc.collect()

        if LOG_TO_WANDB:
            wandb.log({
                f"val/recall@{K}": recall_score,
                f"val/mAP": map_score,
                "train/lr": optimizer.param_groups[0]['lr'],
                "epoch": epoch + 1,
            })
    results['Average mAP'] = sum(results['mAP']) / len(results['mAP'])
    results['Max mAP'] = max(results['mAP'])
    results['Average recall'] = sum(results['recall']) / len(results['recall'])
    results['Max recall'] = max(results['recall'])
    results['params'] = {'embedding_dim': EMBEDDING_DIM,
                         'lr': LR,
                         'weight_decay': WEIGHT_DECAY,
                         'triplet_loss_margin': TRIPLET_LOSS_MARGIN}
    del results['mAP'], results['recall']
    return results

In [23]:
res_1 = train_model(embedding_dim=192,
                    lr=1e-4,
                    weight_decay=1e-5,
                    triplet_loss_margin=0.01,
                    wandb_id=4)
print(res_1)
print('Average mAP:', sum(res_1['mAP']) / len(res_1['mAP']))
print('Max mAP:', max(res_1['mAP']))
print('Average recall:', sum(res_1['recall']) / len(res_1['recall']))
print('Max recall:', max(res_1['recall']))
gc.collect()
torch.cuda.empty_cache()

wandb: Currently logged in as: sergfedchin (xenz5240-higher-school-of-economics) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Training:   0%|          | 0/50 [00:00<?, ?epoch/s]

Epoch 1/50:   0%|          | 0/235 [00:00<?, ?batch/s]

Validation 1/50:   0%|          | 0/3 [00:00<?, ?batch/s]

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

...

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

Filtering recommendations:   0%|          | 0/13097 [00:00<?, ?it/s]

Calculating MAP (no K):   0%|          | 0/45865 [00:00<?, ?it/s]

Calculating Recall@100:   0%|          | 0/45865 [00:00<?, ?it/s]

Recall@100: 0.00266 | mAP: 0.00058
{'mAP': [0.0008679295550692672, 0.0008421959552113947, 0.0006899581197781228, 0.0007631992857623873, 0.0007718949752835239, 0.0006355985823975148, 0.0007658285158874485, 0.0006642931371670496, 0.0006623532979844999, 0.0006783381254331976, 0.0006558617025129341, 0.0006795730259168125, 0.000619387305011439, 0.0006038141918801365, 0.0006408589390837519, 0.0006208596018018727, 0.0006558482115392482, 0.0006153731386369296, 0.0006008049503575892, 0.0005849971075020134, 0.000629402121841812, 0.0005886483699413981, 0.00058348240268023, 0.0005853729873195059, 0.000584342865945305, 0.0005982575548476585, 0.0005499057161558256, 0.0005904289776887774, 0.0006010440914576045, 0.0006239859073034603, 0.0005799938782075454, 0.0005637906181261115, 0.0006087859588797276, 0.0006102253997748657, 0.0005989586067227384, 0.0006273618220482495, 0.000602785516019713, 0.0005835903472736648, 0.0005661874097674361, 0.0006214448571291172, 0.0006074649004466274, 0.00063967330853944

In [30]:
res_1['params'] = {'embedding_dim': 192,
                   'lr': 1e-4,
                   'weight_decay': 1e-5,
                   'triplet_loss_margin': 0.01,
                   'wandb_id': 4}
res_1['Average mAP'] = sum(res_1['mAP']) / len(res_1['mAP'])
res_1['Max mAP'] = max(res_1['mAP'])
res_1['Average recall'] = sum(res_1['recall']) / len(res_1['recall'])
res_1['Max recall'] = max(res_1['recall'])

In [24]:
res_2 = train_model(embedding_dim=192,
                    lr=1e-4,
                    weight_decay=1e-5,
                    triplet_loss_margin=10,
                    wandb_id=5)
print(res_2)
print('Average mAP:', sum(res_2['mAP']) / len(res_2['mAP']))
print('Max mAP:', max(res_2['mAP']))
print('Average recall:', sum(res_2['recall']) / len(res_2['recall']))
print('Max recall:', max(res_2['recall']))
gc.collect()
torch.cuda.empty_cache()

epoch,▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇█████
step,▄█▃▅▂█▂▇▅▄▃▅▂▃▄▇█▆▇▇▅▁▅▂▁▁▄▃█▃▂▂▄▅▂▁▄▇▁▆
train/epoch_loss,█▇▆▆▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/mAP,█▇▄▆▆▆▄▃▄▃▃▂▃▃▃▂▂▃▂▂▂▂▁▂▃▁▂▂▂▃▂▃▂▃▂▃▁▁▃▂
val/recall@100,█▆▃▅▄▅▂▃▃▄▂▃▂▄▂▁▂▂▁▁▃▁▂▃▃▂▂▃▂▂▂▁▂▁▃▂▂▂▂▂
epoch,50
step,220
train/epoch_loss,0.00372
train/loss,0.00369


Training:   0%|          | 0/50 [00:00<?, ?epoch/s]

Epoch 1/50:   0%|          | 0/235 [00:00<?, ?batch/s]

Validation 1/50:   0%|          | 0/3 [00:00<?, ?batch/s]

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

...

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

Filtering recommendations:   0%|          | 0/13097 [00:00<?, ?it/s]

Calculating MAP (no K):   0%|          | 0/45865 [00:00<?, ?it/s]

Calculating Recall@100:   0%|          | 0/45865 [00:00<?, ?it/s]

Recall@100: 0.01493 | mAP: 0.00173
{'mAP': [0.004603079852605875, 0.004291189410687037, 0.005266206900137657, 0.004929010429067197, 0.005101092422051228, 0.004064217557444466, 0.0028931911809765604, 0.0035524608085927105, 0.0026067113725434985, 0.0026011560483109506, 0.0027065865240885043, 0.0029278053915132857, 0.002861522930533587, 0.002533911701469976, 0.0021777564103144, 0.0023210100366373517, 0.002123481557041729, 0.002200687497223048, 0.0020697361773895713, 0.0020813519435512336, 0.0019657861206198502, 0.001992342402771385, 0.001930988683317072, 0.001849483318598129, 0.001903772620715382, 0.0018973022974857322, 0.001849696241316876, 0.0018312616194898395, 0.0017710895778251612, 0.0018415967014753162, 0.0017832916644416063, 0.0017387600395119358, 0.0018456027642756302, 0.0017784638571520176, 0.0017157232823254, 0.001790937456647904, 0.0017042346244773364, 0.0017201486092729079, 0.0016609561347415032, 0.0016430156712505214, 0.001723689799944542, 0.0017575804765494816, 0.00172926531

In [31]:
res_2['params'] = {'embedding_dim': 192,
                   'lr': 1e-4,
                   'weight_decay': 1e-5,
                   'triplet_loss_margin': 10,
                   'wandb_id': 5}
res_2['Average mAP'] = sum(res_2['mAP']) / len(res_2['mAP'])
res_2['Max mAP'] = max(res_2['mAP'])
res_2['Average recall'] = sum(res_2['recall']) / len(res_2['recall'])
res_2['Max recall'] = max(res_2['recall'])

In [25]:
res_3 = train_model(embedding_dim=256,
                    lr=1e-4,
                    weight_decay=1e-5,
                    triplet_loss_margin=0.4,
                    wandb_id=6)
print(res_3)
print('Average mAP:', sum(res_3['mAP']) / len(res_3['mAP']))
print('Max mAP:', max(res_3['mAP']))
print('Average recall:', sum(res_3['recall']) / len(res_3['recall']))
print('Max recall:', max(res_3['recall']))
gc.collect()
torch.cuda.empty_cache()

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇█████
step,█▂▇▇▃▅▆▂▃▃▇▂▂▇▆▂▇█▁▂▄▃▇▅▂▇▄▇▃▄▅▃▄█▅█▂█▃▁
train/epoch_loss,█▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train/loss,█▇▇▆▆▆▆▆▅▄▄▄▃▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
train/lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/mAP,▇▆█▇█▃▅▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/recall@100,▇▆██▇▆▃▅▄▅▅▃▃▂▃▃▂▂▂▂▂▂▂▂▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁
epoch,50
step,220
train/epoch_loss,3.49425
train/loss,3.52356


Training:   0%|          | 0/50 [00:00<?, ?epoch/s]

Epoch 1/50:   0%|          | 0/235 [00:00<?, ?batch/s]

Validation 1/50:   0%|          | 0/3 [00:00<?, ?batch/s]

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

...

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

Filtering recommendations:   0%|          | 0/13097 [00:00<?, ?it/s]

Calculating MAP (no K):   0%|          | 0/45865 [00:00<?, ?it/s]

Calculating Recall@100:   0%|          | 0/45865 [00:00<?, ?it/s]

Recall@100: 0.04189 | mAP: 0.00402
{'mAP': [0.001707288423219029, 0.0016195035266038105, 0.001701003776136881, 0.0014866842251226983, 0.0019075162166324888, 0.0019120102894974226, 0.002224471010828966, 0.002214133127451322, 0.002219178604240468, 0.002065081038283006, 0.002146270277200287, 0.002777132133699997, 0.003226759691582829, 0.0025440922267683987, 0.0026095857508851637, 0.002957168000459878, 0.0026875563250843455, 0.0029886656038368097, 0.003065076301882031, 0.003367527162703818, 0.002875597272197362, 0.003992039818663399, 0.003171476604099169, 0.004785874364696392, 0.0036140554927461617, 0.004392226234637646, 0.003204710436877562, 0.0033830169633466424, 0.0040451701191094825, 0.0036193335273084206, 0.0033014731643132077, 0.00351086008897414, 0.006545671735973895, 0.0038408451571487813, 0.0038067396190967326, 0.003442512675802559, 0.003688774154682213, 0.004935005016207492, 0.0051024037147063914, 0.004000971723607162, 0.0034030293667768655, 0.004482002663020335, 0.00626496273072

In [32]:
res_3['params'] = {'embedding_dim': 256,
                   'lr': 1e-4,
                   'weight_decay': 1e-5,
                   'triplet_loss_margin': 0.4,
                   'wandb_id': 6}
res_3['Average mAP'] = sum(res_3['mAP']) / len(res_3['mAP'])
res_3['Max mAP'] = max(res_3['mAP'])
res_3['Average recall'] = sum(res_3['recall']) / len(res_3['recall'])
res_3['Max recall'] = max(res_3['recall'])

In [26]:
res_4 = train_model(embedding_dim=256,
                    lr=1e-4,
                    weight_decay=1e-5,
                    triplet_loss_margin=1,
                    wandb_id=7)
print(res_4)
print('Average mAP:', sum(res_4['mAP']) / len(res_4['mAP']))
print('Max mAP:', max(res_4['mAP']))
print('Average recall:', sum(res_4['recall']) / len(res_4['recall']))
print('Max recall:', max(res_4['recall']))
gc.collect()
torch.cuda.empty_cache()

epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇█
step,▂▄▅▆▇▇█▃▄▂▁▇▄█▁▇▅▄▁▃▅█▆▂█▂▅█▂▇▃▅▅▇▃▁▁▅▇▄
train/epoch_loss,█▆▅▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
train/loss,█▇▇▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
train/lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/mAP,▁▁▁▁▂▂▂▂▂▃▂▃▃▃▃▃▄▃▆▄▃▄▅▄▄█▄▄▄▄▆▄▄▅█▅▅▄█▅
val/recall@100,▁▁▂▁▂▃▃▁▁▃▃▂▃▂▁▂▆▃▅▄▂▂▄▂▄▆▄▂▄▄▆▃▄▅█▆▆▄▇▅
epoch,50
step,220
train/epoch_loss,0.04056
train/loss,0.0398


Training:   0%|          | 0/50 [00:00<?, ?epoch/s]

Epoch 1/50:   0%|          | 0/235 [00:00<?, ?batch/s]

Validation 1/50:   0%|          | 0/3 [00:00<?, ?batch/s]

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

...

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

Filtering recommendations:   0%|          | 0/13097 [00:00<?, ?it/s]

Calculating MAP (no K):   0%|          | 0/45865 [00:00<?, ?it/s]

Calculating Recall@100:   0%|          | 0/45865 [00:00<?, ?it/s]

Recall@100: 0.01271 | mAP: 0.00154
{'mAP': [0.002053470065009976, 0.002884266632821042, 0.004085015592065355, 0.0031725956331584994, 0.009965717391841537, 0.003286232863605746, 0.0025674346289055096, 0.004253503164492844, 0.0037175606694481327, 0.002071091722660076, 0.0022279818939389836, 0.0018480218046049153, 0.001611815963764639, 0.0024948991054506158, 0.002237010267005994, 0.0018399638620916915, 0.001967639838649446, 0.0017726245569836196, 0.0019011294375347095, 0.0016833364160796578, 0.001637835766657996, 0.001664499384624631, 0.0018262767158771938, 0.001747669790705461, 0.0015807717605072745, 0.0016543686636682205, 0.00176331464446299, 0.0016380606350959875, 0.0016114337453793973, 0.0016680296993030151, 0.0016117831238561442, 0.0015331500054743237, 0.0016400990691523769, 0.0016001453885789068, 0.001694310396028648, 0.001526578401341581, 0.001485478407092518, 0.0014338350676374226, 0.001677710536509993, 0.0015921569744795517, 0.0016287007583166335, 0.0014529991456864154, 0.0015567

In [33]:
res_4['params'] = {'embedding_dim': 256,
                   'lr': 1e-4,
                   'weight_decay': 1e-5,
                   'triplet_loss_margin': 1,
                   'wandb_id': 7}
res_4['Average mAP'] = sum(res_4['mAP']) / len(res_4['mAP'])
res_4['Max mAP'] = max(res_4['mAP'])
res_4['Average recall'] = sum(res_4['recall']) / len(res_4['recall'])
res_4['Max recall'] = max(res_4['recall'])

In [27]:
res_5 = train_model(embedding_dim=256,
                    lr=5e-3,
                    weight_decay=1e-5,
                    triplet_loss_margin=0.4,
                    wandb_id=8)
print(res_5)
print('Average mAP:', sum(res_5['mAP']) / len(res_5['mAP']))
print('Max mAP:', max(res_5['mAP']))
print('Average recall:', sum(res_5['recall']) / len(res_5['recall']))
print('Max recall:', max(res_5['recall']))
gc.collect()
torch.cuda.empty_cache()

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇██
step,▇▁▂▄▇▃▆▇█▂▂▄▇▅█▆▃▅▂▂█▅▄▂▅▇▁▃▅▃▄▇▁▁▆▄▂▃█▇
train/epoch_loss,█▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▅▅▅▅▅▅▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/mAP,▂▂▃▂█▂▃▃▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/recall@100,▂▅▇▅█▅▇▆▄▄▅▄▃▃▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁
epoch,50
step,220
train/epoch_loss,0.08225
train/loss,0.08345


Training:   0%|          | 0/50 [00:00<?, ?epoch/s]

Epoch 1/50:   0%|          | 0/235 [00:00<?, ?batch/s]

Validation 1/50:   0%|          | 0/3 [00:00<?, ?batch/s]

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

...

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

Filtering recommendations:   0%|          | 0/13097 [00:00<?, ?it/s]

Calculating MAP (no K):   0%|          | 0/45865 [00:00<?, ?it/s]

Calculating Recall@100:   0%|          | 0/45865 [00:00<?, ?it/s]

Recall@100: 0.06379 | mAP: 0.02587
{'mAP': [0.002668203463542975, 0.01252117095546736, 0.005889763225754256, 0.005043077547726064, 0.007460525790696567, 0.006889823938261691, 0.011719397742880135, 0.008610499929892759, 0.005831704458253673, 0.014859133326884736, 0.006538045969000802, 0.011815187116058546, 0.005664199601641894, 0.006389990666448679, 0.007925383346712202, 0.010789912881165724, 0.014931867004130675, 0.004098051217855463, 0.0035339638500046042, 0.009929776902068886, 0.008105819397778935, 0.027454647792548678, 0.009584080138163451, 0.016745170700134055, 0.016979887718379817, 0.01677184436516415, 0.008203064739393499, 0.017227031513162135, 0.010707627858522724, 0.027667903210507536, 0.006861908585531028, 0.021382673363736997, 0.025874511744180276, 0.021407083053636488, 0.025022815202653835, 0.029319799816766787, 0.025858139474069643, 0.026109336352166265, 0.028288614061289387, 0.025658240101636108, 0.005091723612452831, 0.015898557782187905, 0.007751315792341516, 0.025229393

In [34]:
res_5['params'] = {'embedding_dim': 256,
                   'lr': 5e-3,
                   'weight_decay': 1e-5,
                   'triplet_loss_margin': 0.4,
                   'wandb_id': 8}
res_5['Average mAP'] = sum(res_5['mAP']) / len(res_5['mAP'])
res_5['Max mAP'] = max(res_5['mAP'])
res_5['Average recall'] = sum(res_5['recall']) / len(res_5['recall'])
res_5['Max recall'] = max(res_5['recall'])

In [28]:
res_6 = train_model(embedding_dim=512,
                    lr=1e-4,
                    weight_decay=1e-5,
                    triplet_loss_margin=0.4,
                    wandb_id=9)
print(res_6)
print('Average mAP:', sum(res_6['mAP']) / len(res_6['mAP']))
print('Max mAP:', max(res_6['mAP']))
print('Average recall:', sum(res_6['recall']) / len(res_6['recall']))
print('Max recall:', max(res_6['recall']))
gc.collect()
torch.cuda.empty_cache()

epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇█
step,▇▄▄▅▄▂▅█▅▇▄▄▂▄▅▂▅█▇▅▂▁▄▃▇▄▇█▅▄▁▇▂▄▂▆▅▆▂█
train/epoch_loss,█▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,██▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/mAP,▁▄▂▂▂▃▃▄▂▃▂▂▃▄▁▃▂▇▃▅▅▂▅▃▂▇▆▇█▇█▇▂▄▂▆█▇█▇
val/recall@100,▁▆▆▇███▆▇▅▆▇▅▅▆▆▆▆▇▇▆▆▆▆▆▆▅▆▅▇▇▇▆▇▆▆▆▆▆▇
epoch,50
step,220
train/epoch_loss,0.01639
train/loss,0.01628


Training:   0%|          | 0/50 [00:00<?, ?epoch/s]

Epoch 1/50:   0%|          | 0/235 [00:00<?, ?batch/s]

Validation 1/50:   0%|          | 0/3 [00:00<?, ?batch/s]

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

...

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

Filtering recommendations:   0%|          | 0/13097 [00:00<?, ?it/s]

Calculating MAP (no K):   0%|          | 0/45865 [00:00<?, ?it/s]

Calculating Recall@100:   0%|          | 0/45865 [00:00<?, ?it/s]

Recall@100: 0.04420 | mAP: 0.00266
{'mAP': [0.001403998230381524, 0.002111406054478256, 0.0024805153847596113, 0.004484363735562427, 0.006750413312393141, 0.01158957473820137, 0.004737712650030377, 0.00679082375378699, 0.011072541411480232, 0.0040234187497112175, 0.002785301149983371, 0.003274132526942994, 0.004386416046047295, 0.006752370938171795, 0.006511174950620508, 0.004888579569605694, 0.0030551326401872148, 0.0049591190927454655, 0.003978870508683251, 0.004026691273129719, 0.003152493148232013, 0.0033817149689012913, 0.007544462704991993, 0.0022865934780891867, 0.007346526375211805, 0.004001401015799749, 0.00467332108055071, 0.004883567553494708, 0.003600518117806404, 0.004384972482448467, 0.0038258231475895577, 0.005401348929999394, 0.004889914218861177, 0.0034342539383767595, 0.0035184714789589385, 0.004155263187519007, 0.003371188887818593, 0.008602950498201525, 0.0028092110119261405, 0.0029515858059042366, 0.0029133583246588003, 0.0027137594541698394, 0.003946670092236878, 

In [35]:
res_6['params'] = {'embedding_dim': 512,
                   'lr': 1e-4,
                   'weight_decay': 1e-5,
                   'triplet_loss_margin': 0.4,
                   'wandb_id': 9}
res_6['Average mAP'] = sum(res_6['mAP']) / len(res_6['mAP'])
res_6['Max mAP'] = max(res_6['mAP'])
res_6['Average recall'] = sum(res_6['recall']) / len(res_6['recall'])
res_6['Max recall'] = max(res_6['recall'])

In [ ]:
for res in sorted([res_1, res_2, res_3, res_4, res_5, res_6], key=lambda r: -r['Max mAP']):
    pprint(res['params'])
    print(f'Average mAP: {res["Average mAP"]:.4f} | Max mAP: {res["Max mAP"]:.4f} | Average recall: {res["Average recall"]:.4f} | Max recall: {res["Max recall"]:.4f}')
    print('-'*83)

{'embedding_dim': 256,
 'lr': 0.005,
 'triplet_loss_margin': 0.4,
 'wandb_id': 8,
 'weight_decay': 1e-05}
Average mAP: 0.0153 | Max mAP: 0.0294 | Average recall: 0.0613 | Max recall: 0.0704
-----------------------------------------------------------------------------------
{'embedding_dim': 512,
 'lr': 0.0001,
 'triplet_loss_margin': 0.4,
 'wandb_id': 9,
 'weight_decay': 1e-05}
Average mAP: 0.0045 | Max mAP: 0.0116 | Average recall: 0.0458 | Max recall: 0.0543
-----------------------------------------------------------------------------------
{'embedding_dim': 256,
 'lr': 0.0001,
 'triplet_loss_margin': 1,
 'wandb_id': 7,
 'weight_decay': 1e-05}
Average mAP: 0.0021 | Max mAP: 0.0100 | Average recall: 0.0198 | Max recall: 0.0482
-----------------------------------------------------------------------------------
{'embedding_dim': 256,
 'lr': 0.0001,
 'triplet_loss_margin': 0.4,
 'wandb_id': 6,
 'weight_decay': 1e-05}
Average mAP: 0.0035 | Max mAP: 0.0065 | Average recall: 0.0331 | Max re

In [ ]:


# params_grid = {'epochs': [25],
#                'embedding_dim': [512, 768, 1024],
#                'lr': [5e-3, 1e-3],
#                'weight_decay': [1e-6, 1e-5, 1e-4],
#                'triplet_loss_margin': [0.3, 0.4, 0.5],
#                'wandb_id': [11]}

params_grid = {'epochs': [25],
               'embedding_dim': [512, 768, 1024],
               'lr': [5e-3, 1e-3],
               'weight_decay': [1e-5],
               'triplet_loss_margin': [0.4],
               'wandb_id': [11]}
grid_results = []
for params_set in [dict(zip(params_grid.keys(), values)) for values in product(*params_grid.values())]:
    # pprint(params_set)
    cur_res = train_model(**params_set)
    grid_results.append(cur_res)

epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇█
step,▁▃▄▅▅▇▄▆▇█▄▇▄▂▄▆▂▅▅▅▅▇▁▄▂▄▅▆▇▄▂▃▇▁▆▁█▅▂▅
train/epoch_loss,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/mAP,▁▂▂▅▇▅▇▄▃▃▇▅▃▅▄▃▃█▂█▅▅▄▄▄▅▃▃▄▃▃▃▃▂▄▆▄▅▄▂
val/recall@100,▁▄▃▇▇▆▇██▇▇▇▇▆▇█▇▇█▄▇█▇▆▇██▇▇▇▄▆▅▆▇█▇▇▇▆
epoch,50
step,220
train/epoch_loss,0.01002
train/loss,0.01076


Training:   0%|          | 0/25 [00:00<?, ?epoch/s]

Epoch 1/25:   0%|          | 0/235 [00:00<?, ?batch/s]

Validation 1/25:   0%|          | 0/3 [00:00<?, ?batch/s]

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

...

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

Filtering recommendations:   0%|          | 0/13097 [00:00<?, ?it/s]

Calculating MAP (no K):   0%|          | 0/45865 [00:00<?, ?it/s]

Calculating Recall@100:   0%|          | 0/45865 [00:00<?, ?it/s]

Recall@100: 0.07404 | mAP: 0.02143


In [53]:
for res in sorted(grid_results + [res_1, res_2, res_3, res_4, res_5, res_6], key=lambda r: -r['Max mAP']):
    pprint(res['params'])
    print(f'Average mAP: {res["Average mAP"]:.4f} | Max mAP: {res["Max mAP"]:.4f} | Average recall: {res["Average recall"]:.4f} | Max recall: {res["Max recall"]:.4f}')
    print('-'*83)

{'embedding_dim': 768,
 'lr': 0.005,
 'triplet_loss_margin': 0.4,
 'weight_decay': 1e-05}
Average mAP: 0.0162 | Max mAP: 0.0308 | Average recall: 0.0644 | Max recall: 0.0796
-----------------------------------------------------------------------------------
{'embedding_dim': 1024,
 'lr': 0.005,
 'triplet_loss_margin': 0.4,
 'weight_decay': 1e-05}
Average mAP: 0.0245 | Max mAP: 0.0307 | Average recall: 0.0643 | Max recall: 0.0781
-----------------------------------------------------------------------------------
{'embedding_dim': 256,
 'lr': 0.005,
 'triplet_loss_margin': 0.4,
 'wandb_id': 8,
 'weight_decay': 1e-05}
Average mAP: 0.0153 | Max mAP: 0.0294 | Average recall: 0.0613 | Max recall: 0.0704
-----------------------------------------------------------------------------------
{'embedding_dim': 512,
 'lr': 0.005,
 'triplet_loss_margin': 0.4,
 'weight_decay': 1e-05}
Average mAP: 0.0138 | Max mAP: 0.0254 | Average recall: 0.0639 | Max recall: 0.0901
-----------------------------------

In [62]:
all_results = grid_results + [res_1, res_2, res_3, res_4, res_5, res_6]
for r in all_results:
    # del r['mAP'], r['recall']
    r['Average recall'] = float(r['Average recall'])
    r['Max recall'] = float(r['Max recall'])
pprint(sorted(all_results, key= lambda r: -r['Max mAP']))

[{'Average mAP': 0.01615796196642287,
  'Average recall': 0.06441866606473923,
  'Max mAP': 0.03076169359405814,
  'Max recall': 0.07962062954902649,
  'params': {'embedding_dim': 768,
             'lr': 0.005,
             'triplet_loss_margin': 0.4,
             'weight_decay': 1e-05}},
 {'Average mAP': 0.024474731302380124,
  'Average recall': 0.06428653001785278,
  'Max mAP': 0.030651082768865377,
  'Max recall': 0.0781470462679863,
  'params': {'embedding_dim': 1024,
             'lr': 0.005,
             'triplet_loss_margin': 0.4,
             'weight_decay': 1e-05}},
 {'Average mAP': 0.015298614830100027,
  'Average recall': 0.06131378561258316,
  'Max mAP': 0.029385527719915552,
  'Max recall': 0.0704326331615448,
  'params': {'embedding_dim': 256,
             'lr': 0.005,
             'triplet_loss_margin': 0.4,
             'wandb_id': 8,
             'weight_decay': 1e-05}},
 {'Average mAP': 0.013806453295130272,
  'Average recall': 0.06387948244810104,
  'Max mAP': 0.0254

In [ ]:
with open('study_results.json', 'w') as f:
    json.dump(all_results, f)

# Optuna study

In [ ]:
def objective(trial: optuna.Trial):
    params = {
        'embedding_dim': trial.suggest_int('embedding_dim', 192, 2048),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
        'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True),
        'triplet_loss_margin': trial.suggest_float('triplet_loss_margin', 0.01, 10)
    }
    print('Trying params:')
    pprint(params)
    cur_res = train_model(epochs=1, wandb_id='optuna', **params)
    cur_res['params'] = params
    with open('study_results.json', 'r') as f:
        previous_runs: list[dict] = json.load(f)
    previous_runs.append(cur_res)
    with open('study_results.json', 'w') as f:
        json.dump(sorted(previous_runs, key=lambda r: -r['Max mAP']), f)

    torch.cuda.empty_cache()
    return cur_res['Max mAP'], cur_res['Average mAP']

# Create a study (minimize or maximize depending on your metric)
study = optuna.create_study(
    directions=['maximize', 'maximize'],
    study_name='dssm_optimization_1',
    storage='sqlite:///optuna_study.db',  # Optional: for persistence
    load_if_exists=True
)

with open('study_results.json', 'r') as f:
    previous_runs = json.load(f)

# Add the previous runs to the study
for run in previous_runs:
    if 'wandb_id' in run['params']:
        del run['params']['wandb_id']
    trial = optuna.trial.create_trial(
        params=run['params'],
        distributions={
            'embedding_dim': optuna.distributions.IntDistribution(192, 2048),
            'lr': optuna.distributions.FloatDistribution(1e-4, 1e-2, log=True),
            'weight_decay': optuna.distributions.FloatDistribution(1e-5, 1e-3, log=True),
            'triplet_loss_margin': optuna.distributions.FloatDistribution(0.01, 10)
        },
        values=(run['Max mAP'], run['Average mAP'])
    )
    study.add_trial(trial)

# Start optimization
study.optimize(func=objective,
               n_trials=50,
               gc_after_trial=True)

# Print best results
print(f"Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-06-06 18:43:45,455] Using an existing study with name 'dssm_optimization' instead of creating a new one.


Trying params:
{'embedding_dim': 535,
 'lr': 0.003850282048867764,
 'triplet_loss_margin': 6.267342319354875,
 'weight_decay': 5.5889585767209694e-05}


wandb: Currently logged in as: sergfedchin (xenz5240-higher-school-of-economics) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Training:   0%|          | 0/1 [00:00<?, ?epoch/s]

Epoch 1/1:   0%|          | 0/235 [00:00<?, ?batch/s]

...

epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▂▂▃▄▄▅▅▆▇▇█
train/epoch_loss,▁
train/loss,█▆▅▄▄▃▃▂▂▂▁▁
train/lr,▁
val/mAP,▁
val/recall@100,▁
epoch,1
step,220
train/epoch_loss,3.36498
train/loss,2.16766


Training:   0%|          | 0/1 [00:00<?, ?epoch/s]

Epoch 1/1:   0%|          | 0/235 [00:00<?, ?batch/s]

[W 2025-06-06 18:46:41,066] Trial 77 failed with parameters: {'embedding_dim': 1310, 'lr': 0.00028194947881181033, 'weight_decay': 2.3185270233966964e-05, 'triplet_loss_margin': 3.8894590577858676} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/serg_fedchn/Homework/6_semester/DL/reelrecs/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_242128/3343878259.py", line 10, in objective
    cur_res = train_model(epochs=1, wandb_id='optuna', **params)
  File "/tmp/ipykernel_242128/3978670567.py", line 66, in train_model
    loss.backward()
  File "/home/serg_fedchn/Homework/6_semester/DL/reelrecs/.venv/lib/python3.10/site-packages/torch/_tensor.py", line 648, in backward
    torch.autograd.backward(
  File "/home/serg_fedchn/Homework/6_semester/DL/reelrecs/.venv/lib/python3.10/site-packages/torch/autograd/__init__.py", line 353, in backward
    _

KeyboardInterrupt: 